# Helpers
Apart from the basic `Mock` objects, the `mock` library brings some useful helper methods/objects which let you make simpler assertions.

## `mock.call` objects
`call()` is a helper object for making simpler assertions for comparing results returned by `call_args`, `call_args_list`, `method_calls` and `method_calls`. 
Examples:

In [3]:
from unittest.mock import call, Mock
m = Mock()
m(1, abc=True)
m.get_food('potato')
assert m.call_args_list == [call(1, abc=True)]
assert m.method_calls == [call.get_food('potato')]
assert m.mock_calls == [call(1, abc=True), call.get_food('potato')]

In [4]:
def get_rows_from_database(database):
    conn = database.connect(dsn='xxx')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM table")
    cursor.fetchall()

m = Mock()
get_rows_from_database(m)

assert m.method_calls == [
    call.connect(dsn='xxx')
]

assert m.mock_calls == [
    call.connect(dsn='xxx'),
    call.connect().cursor(),
    call.connect().cursor().execute('SELECT * FROM table'),
    call.connect().cursor().fetchall()
]

assert m.connect().cursor().method_calls == [
    call.execute('SELECT * FROM table'),
    call.fetchall()
]

## `mock.ANY`
Sometimes you want to make an assertion about **some** of the arguments in a call to mock, but don't care about the others.

In [5]:
from unittest.mock import ANY
import random

def create_user_in_database(database, name):
    metadata = {'name': name, 'number_of_x': 0, 'list_of_favorite_animals': []}
    return database.create_user(name, metadata, x=random.randint(0, 500))

# We want to check that generate_id was called with `name` as the first parameter.
db_mock = Mock()
create_user_in_database(db_mock, 'albert')
create_user_in_database(db_mock, 'potato')
create_user_in_database(db_mock, 'pepe')

# Without ANY:
expected_default_metadata = {'name': 'potato', 'number_of_x': 0, 'list_of_favorite_animals': []}
potato_call = [create_call for create_call in db_mock.create_user.call_args_list if create_call[0][0] == 'potato'][0]
args, kwargs = potato_call
random_x = kwargs['x']
db_mock.create_user.assert_any_call('potato', expected_default_metadata, x=random_x)

# With ANY:
db_mock.create_user.assert_any_call('potato', ANY, x=ANY)

`ANY` is equal to every possible object, so it can be used in comparisons with call lists:

In [6]:
def create_user_in_db_and_maybe_cry(database, name):
    if random.choice((True, False)):
        database.cry()
    else:
        database.happy()
    database.create_user(name)

db_mock = Mock()
create_user_in_db_and_maybe_cry(db_mock, 'albert')
create_user_in_db_and_maybe_cry(db_mock, 'potato')
create_user_in_db_and_maybe_cry(db_mock, 'pepe')

assert db_mock.method_calls == [
    ANY,
    call.create_user('albert'),
    ANY,
    call.create_user('potato'),
    ANY,
    call.create_user('pepe')
]

assert db_mock.create_user.call_args == ANY
assert 'potato' == ANY


## `mock.sentinel` helper
The `sentinel` object provides a convenient way of providing unique objects for your tests.  
Attributes are created on demand as you access them by name. Accessing the same attribute will always return the same object.

In [7]:
from unittest.mock import Mock, sentinel

# Without sentinel - Option 1
m = Mock()
m.method.return_value = my_method_mocked_result = Mock()
assert m.method() == my_method_mocked_result

In [8]:
# Without sentinel - Option 2
m = Mock()
m.method.return_value = 'abcdefg'
assert m.method() == 'abcdefg'

In [9]:
# With sentinel
m = Mock()
m.method.return_value = sentinel.abc
assert m.method() == sentinel.abc

Can be used as a `global` variable in the mocking context.

`sentinel` repr is useful for test failure messages:

In [10]:
print(repr(sentinel.abc))
print(repr(Mock()))

sentinel.abc
<Mock id='139928356933136'>


### `mock.sentinel.DEFAULT`
Pre-created sentinel.

It can be used by side_effect functions to indicate that the normal return value should be used

There are two more "special" sentinels: **`sentinel.MISSING` and `sentinel.DELETED`**, which **should not be used, as they are used for internal implementation of mock library**

## `mock.create_autospec`
Creates a mock object using another object as a spec. 

Additional things you get by using `create_autospec(spec=object)` instead of `Mock(spec=object)`:
- Mocked functions will raise `TypeError` when called incorrectly
- If `spec_set` is set to `True`, attempting to set attributes that don't exist on the spec object will raise an AttributeError.
- It is possible to pass an instance as `spec` and get a instance mock, by setting `instance=True`.  
    Note: `Mock(spec=object)` returns a mock (class mock) whose return_value is a `Mock` (instance mock) with the same spec.

In [11]:
from unittest.mock import Mock, create_autospec
class Dog:
    def get_legs():
        return 4
    def talk(self, message):
        return "wof"

m = Mock(spec=Dog)
m.talk('ok')
m.talk('too', 'many', 'arguments')
m.abc = 5

In [12]:
m2 = create_autospec(Dog)
m2.talk('ok')
try:
    m2.talk('too', 'many', 'args')
except Exception as e:
    print("Failed with {}".format(repr(e)))

Failed with TypeError('too many positional arguments',)


In [13]:
m2 = create_autospec(Dog)
m2.abc = 5

m3 = create_autospec(Dog, spec_set=True)
try:
    m3.abc = 5
except Exception as e:
    print("Failed with {}".format(repr(e)))

Failed with AttributeError("Mock object has no attribute 'abc'",)


## `mock.patch`

Utility to (temporarily) change the value of an object's attribute.

It can be used as a function decorator, class decorator or a context manager. All patchers also have a `start()` and `stop()` method to make it simpler to patch with test's `setUp` and `tearDown`.

In [14]:
from unittest.mock import patch

class ABC:
    def __init__(self):
        self.x = 5
    def method(self):
        return self.x

with patch('__main__.ABC.method', return_value=9) as method_mock:
    print(ABC().method())
    method_mock.assert_called_once()

9


In [15]:
@patch('__main__.ABC.method', return_value=15)
def test_method(method_mock):
    print(ABC().method())
test_method()

15


In [16]:
import unittest

@patch('__main__.ABC.method')
class ABCTests(unittest.TestCase):
    def test_one(abc_mock):
        print("Test One")
        print(abc_mock)
        print(ABC().method)
    
    def test_two(abc_mock):
        print("Test Two")
        print(abc_mock)
        print(ABC().method)

In [17]:
abc = ABC()
patcher = patch('__main__.ABC.method', return_value=30)
print(abc.method())

patcher.start()
print(abc.method())
patcher.stop()

print(abc.method())

5
30
5


### `patch.stop` and `patch.start`
It is common to use `start` in the `setUp` of your TestCase. If an exception is raised in `setUp` after you `patch.start` or `tearDown` fails before you `patch.stop`, it will be a problem.

You have two ways to prevent this:
- `patch.stopall()`: Stops all active patches (only those started with `.start()`)
- Use `self.addCleanup(patcher.stop)` in your `TestCase.setUp`:

In [18]:
class MyTest(unittest.TestCase):
    def setUp():
        patcher = patch('something')
        self.addCleanup(patcher.stop)
        patcher.start()

If you use `unittest.TestCase`, you can also run your test by patching with decorators on every test:

In [19]:
class MyTest(unittest.TestCase):
    def run(self, *args, **kwargs):
        with patch('psycopg2.connect') as self.psycopg2_mock:
            super(self.__class__, self).run(*args, **kwargs)

### `patch` function parameters

- `target`: String in the form of `'package.module.abc.something'`. **First part must be possible to import**  
Mock resolves the `target` string into a "target object" (`package.module.abc`) and an "attribute string"(`"something"`).  
The "target" object is obtained by importing the first part recursively accessing its attributes. If an attribute is not found, it tries to re-import the whole target up to the point it looked up.

In [20]:
patcher = patch('__main__.ABC.nope', return_value=2)  # Does not raise exception until you `start()` it
with patch('__main__.ABC.nope', return_value=2):
    ABC.method()

AttributeError: <class '__main__.ABC'> does not have the attribute 'nope'

- `new`: Defines the new value of the target. By default, it's a new `MagicMock`

In [27]:
import datetime
with patch('datetime.datetime') as date_mock:
    print(date_mock)
    print(datetime.datetime)
with patch('datetime.datetime', new='new value') as date_mock:
    print(date_mock)
    print(datetime.datetime)

<MagicMock name='datetime' id='139928020723528'>
<MagicMock name='datetime' id='139928020723528'>
new value
new value


- `new_callable`: Specify different class or callable object to be called when creating the `new` object. Default: `MagicMock`

In [22]:
with patch('__main__.ABC', new_callable=Mock) as abc_mock:
    print(abc_mock)
with patch('__main__.ABC') as abc_mock:
    print(abc_mock)

<Mock name='ABC' id='139928021022592'>
<MagicMock name='ABC' id='139928021022872'>


- `spec`: Causes patch to pass the object being mocked as the spec  
    Note: When replacing a `class`, the return_value of the created mock will have the same spec.

In [23]:
from utils import get_non_mock_attributes
with patch('__main__.ABC') as abc_mock:
    print(get_non_mock_attributes(abc_mock))
with patch('__main__.ABC', spec=True) as abc_mock:
    print(get_non_mock_attributes(abc_mock))
    print(get_non_mock_attributes(abc_mock()))

[]
['method']
['method']


- `autospec`: With `autospec=True` the mock will be created with a spec from the object being replaced. Like `create_autospec()`.  
You can pass `autospec=some_object` to use another object as the spec.

- `create`: Set to `True` to patch an attribute that does not exist.

In [24]:
print(getattr(ABC, 'nope', None))
with patch('__main__.ABC.nope', create=True) as abc_nope_mock:
    print(ABC.nope)
    print(abc_nope_mock)
print(getattr(ABC, 'nope', None))

None
<MagicMock name='nope' id='139928357161784'>
<MagicMock name='nope' id='139928357161784'>
None


- `**kwargs`: Set attributes on the created mock

In [25]:
ABC.one = 1
with patch('__main__.ABC', one='UNO', two=2):
    print(ABC)
    print(ABC.one)
    print(ABC.two)
print(ABC)
print(ABC.one)
print(hasattr(ABC, 'two'))

<MagicMock name='ABC' id='139928366382608'>
UNO
2
<class '__main__.ABC'>
1
False


## `patch.object`
Patch an object's attribute.

The only difference with `patch` is the way you specify the `target`.

In [26]:
with patch.object(ABC, 'method') as abc_method_mock:
    print(ABC.method)
    print(abc_method_mock)

<MagicMock name='method' id='139928021021024'>
<MagicMock name='method' id='139928021021024'>


## `patch.dict`
Patch a dictionary or dict-like object.

In [125]:
mydict = {'a': 'AAA', 'b': 'BBB'}
with patch.dict(mydict, {'b': 'newB', 'c': 'newC'}):
    print(mydict)
with patch.dict(mydict, b='newB', c='newC'):
    print(mydict)
print(mydict)

{'a': 'AAA', 'b': 'newB', 'c': 'newC'}
{'a': 'AAA', 'b': 'newB', 'c': 'newC'}
{'a': 'AAA', 'b': 'BBB'}


- `clear` parameter: Dictionary will be cleared before new values are set.

In [124]:
with patch.dict(mydict, {'b': 'newB', 'c': 'newC'}, clear=True):
    print(mydict)
print(mydict)

{'b': 'newB', 'c': 'newC'}
{'a': 'AAA', 'b': 'BBB'}


## `patch.multiple`
Peform multiple patches in a single call. Takes the target to be patched (string or object) and kwargs for patches:

In [24]:
from unittest.mock import DEFAULT
with patch.multiple(ABC, method=9, something=DEFAULT, create=True) as values:
    print(ABC())
    print(ABC().method)
    print(values)

9
{'something': <MagicMock name='something' id='140083245287688'>}


In [26]:
with patch.multiple('__main__', ABC=DEFAULT, newvariable=9, create=True) as values:
    print(ABC())
    print(newvariable)
    print(values)

<MagicMock name='ABC()' id='140083245518920'>
9
{'ABC': <MagicMock name='ABC' id='140083245314232'>}
